In [ ]:
import numpy as np
from pandas import read_csv, DataFrame
from keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
import seaborn as sns
from sklearn.model_selection import train_test_split

path_to_model = r'C:/bauman/git_bauman/ebw_model'
path_to_data  = r"C:/bauman/git_bauman/data/ebw_data.csv"

dataset = pd.read_csv(path_to_data)
#dataset.head()
#dataset.describe()

# посмотрели есть ли выпадения данных
print('percentage of NaN cells:')
print(dataset.isna().sum() / dataset.shape[0] *100)


# нормализация и запоминание мин и макс по столбцам
maxdataset = dataset[['IW','IF','VW','FP','Depth','Width']].max()
mindataset = dataset[['IW','IF','VW','FP','Depth','Width']].min()

maxIW = maxdataset['IW'] 
minIW = mindataset['IW'] 
normIW = (dataset['IW'] - minIW ) / (maxIW - minIW )

maxIF = maxdataset['IF'] 
minIF = mindataset['IF'] 
normIF = (dataset['IF'] - minIF ) / (maxIF - minIF )

maxVW = maxdataset['VW'] 
minVW = mindataset['VW'] 
normVW = (dataset['VW'] - minVW ) / (maxVW - minVW )

maxFP = maxdataset['FP'] 
minFP = mindataset['FP'] 
normFP = (dataset['FP'] - minFP ) / (maxFP - minFP )

maxDepth = maxdataset['Depth'] 
minDepth = mindataset['Depth'] 
normDepth = (dataset['Depth'] - minDepth ) / (maxDepth - minDepth )

maxWidth = maxdataset['Width'] 
minWidth = mindataset['Width'] 
normWith = (dataset['Width'] - minWidth ) / (maxWidth - minWidth )

minmax = pd.DataFrame({'maxDepth': maxDepth,
                       'minDepth': minDepth,
                       'minWidth': minWidth,
                       'maxWidth': maxWidth,
                       'minIW': minIW,
                       'maxIW': maxIW,
                       'minIF': minIF,
                       'maxIF': maxIF,
                       'minVW': minVW,
                       'maxVW': maxVW,
                       'minFP': minFP,
                       'maxFP': maxFP,
                        }, index=[0])
minmax.to_csv(path_to_model + r'/minmax.csv')

normdataset = pd.DataFrame({'IW': normIW, 
                            'IF': normIF,
                            'VW': normVW,
                            'FP': normFP,
                            'Depth': normDepth,
                            'Width': normWith,
                            })

# переменные для входных и выходных данных
X = normdataset[['IW','IF','VW','FP']]
Y = normdataset[['Depth','Width']] 

# делим выборку и перемешиваем
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.05, shuffle = True)

# создаем модель
model = Sequential()
model.add(Dense(4, activation = 'sigmoid'))
model.add(Dense(2, activation = 'sigmoid'))
model.compile( loss = 'mse',optimizer = 'adam', metrics = ['accuracy'] )

history = model.fit(X_train,Y_train, epochs = 1000, validation_data = (X_train,Y_train), verbose = 1)

model.save(path_to_model)

# для инфомации выводит кривую обучения
plt.plot(history.history['accuracy'],label = 'Обучающая выборка')
plt.plot(history.history['val_accuracy'],label = 'Точность тест')
plt.title('Обучение')
plt.xlabel('эпохи')
plt.ylabel(model.loss)

In [3]:
# делаем предсказание
# данный код будет выполняться на сервере
import keras
import pandas as pd

path_to_model = r'C:/bauman/git_bauman/ebw_model'

human_input = pd.DataFrame({'IW': [47],
                            'IF': [134],
                            'VW': [10],
                            'FP': [110]})


minmax = pd.read_csv(path_to_model + r'/minmax.csv')

human_input['IW'] = (human_input['IW'] - minmax.minIW) / (minmax.maxIW - minmax.minIW)
human_input['IF'] = (human_input['IF'] - minmax.minIF) / (minmax.maxIF - minmax.minIF)
human_input['VW'] = (human_input['VW'] - minmax.minVW) / (minmax.maxVW - minmax.minVW)
human_input['FP'] = (human_input['FP'] - minmax.minFP) / (minmax.maxFP - minmax.minFP)


model= keras.models.load_model(path_to_model)

mypredcit = model.predict(human_input)

predict = pd.DataFrame({'Depth': mypredcit[:,0]*(minmax.maxDepth-minmax.minDepth)+minmax.minDepth, 
                        'Width': mypredcit[:,1]*(minmax.maxWidth-minmax.minWidth)+minmax.minWidth})

# тестовые значение для ответа: 0.92	1.86
 
print(predict)

#print("Predicted Depth: ",predict.Depth)
#print("Predicted Width: ",predict.Width)


1/1 [==============================] - 0s 34ms/step
      Depth     Width
0  1.133488  1.923074
